<a href="https://colab.research.google.com/github/Flaviaestat/Phyton/blob/master/API_Bing_image_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building an image dataset

Vamos usar o [Bing Image Search API]() da Microsoft para buscar as imagens.
Precisamos criar uma conta para pegar a chave da API

Podemos entrar no [link](https://azure.microsoft.com/en-us/try/cognitive-services/my-apis/) para pegar a chave da API

In [0]:
from requests import exceptions
import requests
import cv2
import os

In [23]:
#para salvar os arquivos no google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
query = 'feliz'

folder = '/dataset/feliz'

In [0]:
#caminho drive pessoal
workdir_path = '/content/drive/My Drive/' + '00 PUC BI MASTER/08 - LUI'

path = workdir_path + folder

Vamos pegar as primeiras 250 imagens. Porém vamos pegar 50 por request. Vamos fazer uma consulta paginada

Precisamos olhar a [documentação do endpoint](https://docs.microsoft.com/en-us/rest/api/cognitiveservices-bingsearch/bing-images-api-v7-reference) para entender como funciona


[key](https://docs.microsoft.com/pt-br/azure/cognitive-services/bing-image-search/image-sdk-python-quickstart)

In [0]:
API_KEY = '2d677ffb9a504ccba024437f4f9b1e7f'
MAX_RESULTS = 250
GROUP_SIZE = 50

URL = "https://api.cognitive.microsoft.com/bing/v7.0/images/search"

Ao baixar imagens da internet Python e a biblioteca requests podem lançar e exceções. Vamos definir algumas para poder tratá-las de maneira adequado quando e se ocorrerem

In [0]:
EXCEPTIONS = set([IOError, FileNotFoundError,
                  exceptions.RequestException, exceptions.HTTPError,
                  exceptions.ConnectionError, exceptions.Timeout])

Vamos setar alguns parametros da consulta

In [0]:
headers = {'Ocp-Apim-Subscription-Key': API_KEY}

In [0]:
params = {'q': query, 'offset': 0, 'count': GROUP_SIZE}

Vamos fazer uma requisição para a API 

In [0]:
search = requests.get(URL, headers=headers, params=params)
search.raise_for_status()

In [0]:
results = search.json()

In [0]:
estimated_num_results = min(results['totalEstimatedMatches'], MAX_RESULTS)

In [19]:
estimated_num_results

250

In [0]:
total = 0

In [29]:
for offset in range(0, estimated_num_results, GROUP_SIZE):
    print("Fazendo requisições para o grupo {}-{} de {}...".format(offset, offset + GROUP_SIZE,
                                                                   estimated_num_results))
    params['offset'] = offset
    search = requests.get(URL, headers=headers, params=params)
    search.raise_for_status()
    
    results = search.json()
    print("Salvando imagens para o grupo {}-{} de {}...".format(offset, offset + GROUP_SIZE,
                                                                estimated_num_results))
    for v in results['value']:
        try:
            print("Buscando: {}".format(v['contentUrl']))
            r = requests.get(v['contentUrl'], timeout=30)
            
            ext = v['contentUrl'][v['contentUrl'].rfind('.'):]
            path = os.path.sep.join([output, "{}{}".format(str(total).zfill(8), ext)])
            print(path)
            f = open(path, 'wb')
            f.write(r.content)
            f.close()
            
        except Exception as e:
            if type(e) in EXCEPTIONS:
                print('Pulando: {}'.format(v['contentUrl']))
                continue
                
        image = cv2.imread(path)
        
        if image is None:
            print('Deletando: {}'.format(path))
            #os.remove(path)
            continue
        
        total += 1

Fazendo requisições para o grupo 0-50 de 250...
Salvando imagens para o grupo 0-50 de 250...
Buscando: http://i.ytimg.com/vi/sigge8kJ9wg/maxresdefault.jpg
dataset/feliz/00000000.jpg
Pulando: http://i.ytimg.com/vi/sigge8kJ9wg/maxresdefault.jpg
Buscando: https://i.ytimg.com/vi/Alc08yQ2lZg/maxresdefault.jpg
dataset/feliz/00000000.jpg
Pulando: https://i.ytimg.com/vi/Alc08yQ2lZg/maxresdefault.jpg
Buscando: https://img.michaels.com/L6/3/IOGLO/887663939/221500641/28181_27801.jpg?fit=inside%7C1024:1024
dataset/feliz/00000000.jpg?fit=inside%7C1024:1024
Pulando: https://img.michaels.com/L6/3/IOGLO/887663939/221500641/28181_27801.jpg?fit=inside%7C1024:1024
Buscando: https://upload.wikimedia.org/wikipedia/commons/thumb/3/34/Neftal%C3%AD_Feliz_on_July_31%2C_2015.jpg/1200px-Neftal%C3%AD_Feliz_on_July_31%2C_2015.jpg
dataset/feliz/00000000.jpg
Pulando: https://upload.wikimedia.org/wikipedia/commons/thumb/3/34/Neftal%C3%AD_Feliz_on_July_31%2C_2015.jpg/1200px-Neftal%C3%AD_Feliz_on_July_31%2C_2015.jpg
Bu